In [128]:
import config
import json
import requests
import vulture_scraper as vulture
api_key = config.API_key
import time 
import datetime

In [4]:
def search_movies_timed(api_key, list_of_movies):
        lengthToSleep = 40/59
        
        data = []
        
        for movie in list_of_movies:
            url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
            data.append(requests.get(url).json())
            time.sleep(lengthToSleep)
            
        results_list_nested = [x.get('results') for x in data]
        results = [val for sublist in results_list_nested if type(sublist) == list for val in sublist] #<-- if clause makes sure that we're not iterating over none
        
        num_results = sum(filter(None,[x.get('total_results') for x in data])) # <--- not super useful but i was proud of the code
        
        return results 
        

In [5]:
def get_movies_timed(api_key,ids):
    
    lengthToSleep = 40/59        
    
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        data.append(requests.get(url).json())
        
        time.sleep(lengthToSleep)
        
    return data

In [7]:
# input list of movies names, return full info for movies
def search_and_get(api_key,list_of_movies):
    search = search_movies_timed(api_key,list_of_movies)
    
    ids = [d['id'] for d in search]
    
    return get_movies_timed(api_key,ids)
        

In [8]:
hulu_results = search_and_get(api_key, vulture.best_hulu)


In [175]:
hulu_results[0:1]

[{'adult': False,
  'backdrop_path': '/c5g1Dn1tF22CS2oOvHDNKr1Ve2U.jpg',
  'belongs_to_collection': None,
  'budget': 4500000,
  'genres': [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}],
  'homepage': 'http://www.requiemforadream.com/',
  'id': 641,
  'imdb_id': 'tt0180093',
  'original_language': 'en',
  'original_title': 'Requiem for a Dream',
  'overview': 'The hopes and dreams of four ambitious people are shattered when their drug addictions begin spiraling out of control. A look into addiction and how it overcomes the mind and body.',
  'popularity': 8.015,
  'poster_path': '/nOd6vjEmzCT0k4VYqsA2hwyi87C.jpg',
  'production_companies': [{'id': 2188,
    'logo_path': None,
    'name': 'Artisan Entertainment',
    'origin_country': 'US'}],
  'production_countries': [{'iso_3166_1': 'US',
    'name': 'United States of America'}],
  'release_date': '2000-10-06',
  'revenue': 7390108,
  'runtime': 102,
  'spoken_languages': [{'iso_639_1': 'en', 'name': 'English'}],
  'status'

In [38]:
## key:type(value) test for list of dictionaries
## nifty lil' function to check the data types of dictionary values
def key_test(list_of_dicts):
    return [{key:type(d.get(key)) for key in d.keys()} for d in list_of_dicts]          

In [60]:
k_typeValue = key_test(hulu_results)
k_typeValue[0]


{'adult': bool,
 'backdrop_path': str,
 'belongs_to_collection': NoneType,
 'budget': int,
 'genres': list,
 'homepage': str,
 'id': int,
 'imdb_id': str,
 'original_language': str,
 'original_title': str,
 'overview': str,
 'popularity': float,
 'poster_path': str,
 'production_companies': list,
 'production_countries': list,
 'release_date': str,
 'revenue': int,
 'runtime': int,
 'spoken_languages': list,
 'status': str,
 'tagline': str,
 'title': str,
 'video': bool,
 'vote_average': float,
 'vote_count': int}

In [182]:
## search results will be a list of dictionaries 
def parse_function(search_results):
    results = []
    for d in search_results:
        if (d['imdb_id'] != '' and d['imdb_id'] != None and d['status'] == 'Released'):
            parsed_results = {}

            parsed_results['id'] = d.get('id')
            parsed_results['original_language'] = d['original_language']
            parsed_results['overview'] = d.get('overview')
            parsed_results['title'] = d.get('title')
            parsed_results['popularity'] = d.get('popularity')

            if d['belongs_to_collection'] == None:
                parsed_results['collection'] = None
            else: 
                parsed_results['collection'] = d['belongs_to_collection']['name']

            parsed_results['budget'] = d.get('budget')
            parsed_results['revenue'] = d.get('revenue')

            if len(d['genres']) == 0:
                parsed_results['genres'] = None
            else:
                parsed_results['genres'] = [x['name'] for x in d['genres']]


            if len(d['production_companies']) == 0:
                parsed_results['production_company'] = None
            else:
                parsed_results['production_company'] = [x['name'] for x in d['production_companies']]

            if len(d['production_countries']) == 0:
                parsed_results['production_countries'] = None
            else:
                parsed_results['production_countries'] = [x['name'] for x in d['production_countries']]

            if d['release_date'] == '':
                parsed_results['release_date'] = None 
            else:
                parsed_results['release_date'] = datetime.datetime.strptime(d['release_date'],"%Y-%m-%d").date()

            if d['runtime'] == None:
                parsed_results['run_time'] = None
            else:
                parsed_results['run_time'] = d['runtime']
           
            if len(d['spoken_languages']) == 0:
                parsed_results['spoken_lang'] = None 
            else:
                parsed_results['spoken_lang'] = [x['name'] for x in d['spoken_languages']]
            
            if d['tagline'] == '':
                parsed_results['tagline'] = None
            else: 
                parsed_results['tagline'] = d['tagline']
            
            parsed_results['vote_avg'] = d['vote_average']
            parsed_results['vote_count'] = d['vote_count']
            parsed_results['imdb_id'] = d['imdb_id']
            parsed_results['vote_count'] = d['vote_count']
            
            
            results.append(parsed_results)
    return results

In [183]:
parsed_hulu_results = parse_test(hulu_results)
parsed_hulu_results

[{'id': 641,
  'original_language': 'en',
  'overview': 'The hopes and dreams of four ambitious people are shattered when their drug addictions begin spiraling out of control. A look into addiction and how it overcomes the mind and body.',
  'title': 'Requiem for a Dream',
  'popularity': 8.015,
  'collection': None,
  'budget': 4500000,
  'revenue': 7390108,
  'genres': ['Crime', 'Drama'],
  'production_company': ['Artisan Entertainment'],
  'production_countries': ['United States of America'],
  'release_date': datetime.date(2000, 10, 6),
  'run_time': 102,
  'spoken_lang': ['English'],
  'tagline': None,
  'vote_avg': 8.0,
  'vote_count': 5401,
  'imdb_id': 'tt0180093',
  'review_count': 5401},
 {'id': 581,
  'original_language': 'en',
  'overview': "Wounded Civil War soldier, John Dunbar tries to commit suicide—and becomes a hero instead. As a reward, he's assigned to his dream post, a remote junction on the Western frontier, and soon makes unlikely friends with the local Sioux tri

In [177]:
key_test(parsed_hulu_results)

[{'id': int,
  'original_language': str,
  'overview': str,
  'title': str,
  'popularity': float,
  'collection': NoneType,
  'budget': int,
  'revenue': int,
  'genres': list,
  'production_company': list,
  'production_countries': list,
  'release_date': datetime.date,
  'run_time': int,
  'spoken_lang': list,
  'tagline': NoneType,
  'vote_avg': float,
  'vote_count': int,
  'imdb_id': str,
  'review_count': int},
 {'id': int,
  'original_language': str,
  'overview': str,
  'title': str,
  'popularity': float,
  'collection': NoneType,
  'budget': int,
  'revenue': int,
  'genres': list,
  'production_company': list,
  'production_countries': list,
  'release_date': datetime.date,
  'run_time': int,
  'spoken_lang': list,
  'tagline': str,
  'vote_avg': float,
  'vote_count': int,
  'imdb_id': str,
  'review_count': int},
 {'id': int,
  'original_language': str,
  'overview': str,
  'title': str,
  'popularity': float,
  'collection': NoneType,
  'budget': int,
  'revenue': int,
 